In [1]:
using CSV, DataFrames, Random, HTTP, Distributions

In [2]:
url = "https://raw.githubusercontent.com/ArieBeresteanu/Bounds.jl/main/Fortran/Chtdata.csv"

http_response = HTTP.get(url)
file = CSV.File(http_response.body,header=false)

cht = DataFrame(file)

rename!(cht, :Column1 => :yl, :Column2 => :yu, :Column3 => :edu)
first(cht,3)

,yl,yu,edu
,Float64,Float64,Int64
1,4.0,4.0969,28
2,4.5441,4.699,28
3,4.1761,4.301,34


In [3]:
s = describe(cht, :mean, cols=[:yl,:yu])
H0 = [s[1,2], s[2,2]]

2-element Vector{Float64}:
 4.434663145221974
 4.967426358164033

In [6]:
include("C:\\Users\\The Group Leader\\Notebooks\\Research\\Bounds.jl-1\\src\\setBLP.jl")

Main.setBLP

In [7]:
setBLP.EY(cht[!,:yl],cht[!,:yu],H0)

LoadError: BoundsError: attempt to access 1-element Vector{Float64} at index [1900]

In [11]:
plus(x::Real)=max(0.0,x)
minus(x::Real)=max(0.0,-x)

minus (generic function with 1 method)

In [21]:
σ = 0.25
Pi = [1.0 σ; σ 1.21] #covariance matrix for yl,yu
sqrt_n = 10
LB,UB = H0

d = MvNormal([0, 0],Pi) #defining the joint normal distribution
B = 2000 #number of MC iterations to compute the critical value
α = 0.95  #confidence level for the critical value1

## Following Algorithm on page 780 in BM2008:
rr = (rand(d,B)); #drawing B pairs from a bivariate-normal distribution.

## test based on Hausdorff distance:
r_H = maximum(abs.(rr),dims=1);
sort!(r_H,dims=2)
c_H = r_H[floor(Int64,α*B)]
CI_H = [LB-c_H/sqrt_n,UB+c_H/sqrt_n]


#test based on directed Hausdorff distance:
r_dH = maximum([plus.(rr[1,:]) minus.(rr[2,:])],dims=2)
sort!(r_dH,dims=1)
c_dH = r_dH[floor(Int64,α*B)]
CI_dH = [LB-c_dH/sqrt_n,UB+c_dH/sqrt_n]
dHtest = testResults(testStat_dH,c_dH,CI_dH)

LoadError: UndefKeywordError: keyword argument dims not assigned

In [13]:
size(rr)

(2, 2000)

In [14]:
size(r_H)

(1, 2000)

In [16]:
typeof(r_H)

Matrix{Float64} (alias for Array{Float64, 2})

In [17]:
size(r_dH)

(1,)

In [27]:
k = maximum([plus.(rr[1,:]) minus.(rr[2,:])],dims=2)
sort!(k,dims=1)

2000×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 2.8427004685518007
 2.9203794846414604
 2.9337797305236064
 2.995705490745582
 3.100997070100628
 3.125232403907023
 3.171814499008547
 3.176882391013481
 3.2808335857403694
 3.283920336607822
 3.3737426455009274
 3.409149683677947

In [25]:
size(k)

(2000, 1)